In [1]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np
from time import time

In [2]:
df = pd.read_csv("../beyazperde dataset/movies2.csv")
df = df.assign(birleşim=[str(f'''{df["özet"][i]} {str(df["oyuncular"][i]).strip("[]").replace("'", "").replace(",",".")}. {str(df["yönetmen"][i]).strip("[]").replace("'", "").replace(",",".")}. {str(df["tür"][i]).strip("[]").replace("'", "").replace(",",".")}''').lower() + "." for i in range(len(df["başlık"]))])

descriptions = np.array(df["birleşim"])
titles = np.array(df["başlık"])

In [3]:
models = []
model1 = SentenceTransformer("sentence-transformers/distilbert-multilingual-nli-stsb-quora-ranking")
model2 = SentenceTransformer("sentence-transformers/paraphrase-xlm-r-multilingual-v1")
models.append(model1)
models.append(model2)

In [ ]:
def embed(models, descriptions):
    embeddings = []
    for idx in range(len(models)):
        start = time()
        embedding = models[idx].encode(descriptions)
        end = time()
        print(f"Embedding time of model {idx+1}: {end-start}")
        embeddings.append(embedding)
    return embeddings

corpus_embeddings = embed(models, descriptions)

In [37]:
def search(queries, corpus_embeddings, models, titles, top_k):
    for idx in range(len(models)):
        for query in queries:
            query_embedding = models[idx].encode(query)
            scores = np.array(util.cos_sim(query_embedding, corpus_embeddings[idx])).reshape(-1)
            results = sorted(zip(titles, scores), key = lambda x: x[1])[::-1]
            print(f"Model {idx+1} - Query: {query}\n")
            for i in results[:top_k]:
                print(f"{i[0]}: {i[1]}")
            print("\n")

queries = ["Her biri farklı süper güçlere sahip olan aile bireylerinin normal yaşama ayak uyduramayışı ve kahramanlık yaparken yaşadıklarını anlatan animasyon filmi.",
           "Mavi yaratıkların bulunduğu gezegeni sömüren insanlar ile mavi yaratıkların savaşını anlatan, james cameronun yönettiği film",
           "Sihirli bir ailenin sihirli olan evinin yıkılması ve sihirsiz olduğuna inanılan aile üyesinin tüm aileyi biraraya getirmeyi başararak evi yeniden inşaları",
           "Maceralara atılmak isteyen bir balığın kaybolmasını ve bu süreçte yaşadığı maceraları anlatan animasyon filmi",
           "liseye geçen süper güçleri olan çocukları anlatan film. Babası ve annes süper güçlü olan çocuğun sonradan süper güçler kazanmasını anlatıyor.",
           "Quentin Tarantino imzalı doğrusal olmayan bir zaman akışına sahip, iki gangsterin maceralarını konu alan suç ve gerilim filmi."
          ]
search(queries, corpus_embeddings, models, titles, 10)

Model 1 - Query: Her biri farklı süper güçlere sahip olan aile bireylerinin normal yaşama ayak uyduramayışı ve kahramanlık yaparken yaşadıklarını anlatan animasyon filmi.

İnanılmaz Aile 2: 0.9383630752563477
İnanılmaz Aile: 0.9250811338424683
Modern Çağın Samurayları: 0.9208818078041077
Yüzüklerin Efendisi: İki Kule: 0.9202781915664673
Tehlikeli Oyun: 0.9179083108901978
Dünyalar Savaşı: 0.9158708453178406
Kahramanlar Okulu: 0.9146226048469543
Arkadaş: 0.9142215847969055
Rango: 0.9129145741462708
Yazı Tura: 0.9127618074417114


Model 1 - Query: Mavi yaratıkların bulunduğu gezegeni sömüren insanlar ile mavi yaratıkların savaşını anlatan, james cameronun yönettiği film

Prometheus: 0.9488359689712524
Yasak Bölge 9: 0.9444435834884644
Riddick: 0.9416180849075317
Yaratık 2: 0.9396702647209167
They Live: 0.9381137490272522
Avatar: 0.9373224377632141
Patton: 0.93617182970047
Ender's Game: Uzay Oyunları: 0.9360252022743225
Titanik: 0.9352249503135681
İki Yüzlü: 0.9337202906608582


Model 1 - 